In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
cols = ['user','item','rating','timestamp']

train = pd.read_csv('data/ua.base',delimiter='\t',names = cols)
test = pd.read_csv('data/ua.test',delimiter='\t',names = cols)

In [3]:
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import csr

def vectorize_dic(dic,ix=None,p=None,n=0,g=0):
    """
    dic -- dictionary of feature lists. Keys are the name of features
    ix -- index generator (default None)
    p -- dimension of feature space (number of columns in the sparse matrix) (default None)
    """
    if ix==None:
        ix = dict()

    nz = n * g

    col_ix = np.empty(nz,dtype = int)

    i = 0
    for k,lis in dic.items():
        for t in range(len(lis)):
            ix[str(lis[t]) + str(k)] = ix.get(str(lis[t]) + str(k),0) + 1
            col_ix[i+t*g] = ix[str(lis[t]) + str(k)]
        i += 1

    row_ix = np.repeat(np.arange(0,n),g)
    data = np.ones(nz)
    if p == None:
        p = len(ix)

    ixx = np.where(col_ix < p)
    return csr.csr_matrix((data[ixx],(row_ix[ixx],col_ix[ixx])),shape=(n,p)),ix


def batcher(X_, y_=None, batch_size=-1):
    n_samples = X_.shape[0]

    if batch_size == -1:
        batch_size = n_samples
    if batch_size < 1:
       raise ValueError('Parameter batch_size={} is unsupported'.format(batch_size))

    for i in range(0, n_samples, batch_size):
        upper_bound = min(i + batch_size, n_samples)
        ret_x = X_[i:upper_bound]
        ret_y = None
        if y_ is not None:
            ret_y = y_[i:i + batch_size]
            yield (ret_x, ret_y)

In [4]:
x_train,ix = vectorize_dic({'users':train['user'].values,
                            'items':train['item'].values},n=len(train.index),g=2)

x_test,ix = vectorize_dic({'users':test['user'].values,
                           'items':test['item'].values},ix,x_train.shape[1],n=len(test.index),g=2)

y_train = train['rating'].values
y_test = test['rating'].values

x_train = x_train.todense()
x_test = x_test.todense()


In [5]:
n,p = x_train.shape

# length of embedding vector
k = 10 

x = tf.placeholder('float',[None,p])
y = tf.placeholder('float',[None,1])

w0 = tf.Variable(tf.zeros([1]))
w = tf.Variable(tf.zeros([p]))
v = tf.Variable(tf.random_normal([k,p],mean=0,stddev=0.01))

In [6]:
linear_term = tf.add(w0, tf.reduce_sum(tf.multiply(w,x), axis=1, keepdims=True))

In [7]:
interaction_term = 0.5 * tf.reduce_sum(
    tf.subtract(
        tf.pow(tf.matmul(x, tf.transpose(v)), 2),
        tf.matmul(tf.pow(x,2), tf.transpose(tf.pow(v, 2)))
    ), axis=1, keepdims=True)

In [8]:
y_hat = tf.add(linear_term, interaction_term)

In [13]:
lambda_w = tf.constant(0.001, name='lambda_w')
lambda_v = tf.constant(0.001, name='lambda_v')

l2_norm = tf.reduce_sum(
    tf.add(
        tf.multiply(lambda_w,tf.pow(w,2)),
        tf.multiply(lambda_v,tf.pow(v,2))
    )
)

In [14]:
error = tf.reduce_mean(tf.square(y-y_hat))
loss = tf.add(error, l2_norm)
train_op = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(loss)

In [31]:
epochs = 10
batch_size = 10000

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(epochs):
        perm = np.random.permutation(x_train.shape[0])
        losses = []
        for bX, bY in batcher(x_train[perm], y_train[perm], batch_size):
            _, t = sess.run([train_op, loss], feed_dict={x: bX, y: bY.reshape(-1, 1)})
            losses.append(t)
        losses = np.array(losses)
        print('Loss=%f' % losses.mean())
    
    errors = []
    for bX, bY in batcher(x_test, y_test):
        errors.append(sess.run(error, feed_dict={x: bX, y: bY.reshape(-1, 1)}))
    RMSE = np.sqrt(np.array(errors).mean())
    print ('RMSE=%f' % RMSE)
        
        

Loss=11.679044
Loss=8.141493
Loss=5.802639
Loss=4.271319
Loss=3.261834
Loss=2.584970
Loss=2.141384
Loss=1.841594
Loss=1.651988
Loss=1.526650
RMSE=1.241742


In [25]:
generator = batcher(x_train, y_train, 100)
bx, by = next(generator)

In [30]:
by.reshape(-1, 1).shape

(100, 1)